In [1]:
# importing the libraries
import numpy as np

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

import pickle
def load_data(in_dir):
    f = open(in_dir,'rb')
    train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid = pickle.load(f)
    return train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid

data_path = 'adress.pkl'
checkpoint = 'checkpoint/'

train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid = load_data(data_path)

# converting training images into torch format
train_x = train_data
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_label
train_y = train_y.reshape(2379).astype(float);
train_y = torch.from_numpy(train_y)


# shape of training data
# train_x.shape, train_y.shape
from torch.utils.data import TensorDataset, DataLoader

# my_x = [np.array([[1.0,2],[3,4]]),np.array([[5.,6],[7,8]])] # a list of numpy arrays
# my_y = [np.array([4.]), np.array([2.])] # another list of numpy arrays (targets)

# tensor_x = torch.Tensor(my_x) # transform to torch tensor
# tensor_y = torch.Tensor(my_y)

CTX = torch.device('cuda')

train_dataset = TensorDataset(train_x.to(CTX),train_y.to(CTX)) # create your datset

 # create your dataloader


In [2]:
# converting training images into torch format
val_x = valid_data
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = valid_label
val_y = val_y.reshape(297).astype(float);
val_y = torch.from_numpy(val_y)
# shape of training data

val_dataset = TensorDataset(val_x,val_y) # create your datset

In [3]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

batch_size = 64
val_size = 297
# train_size = train_x.size(0) - val_size 

# train_data,val_data = random_split(dataset,[train_size,val_size])
# print(f"Length of Train Data : {len(train_data)}")
# print(f"Length of Validation Data : {len(val_data)}")

#output
#Length of Train Data : 2379
#Length of Validation Data : 297

#load the train and validation into batches.
train_dl = DataLoader(train_dataset,batch_size, shuffle = True, num_workers = 0)
val_dl = DataLoader(val_dataset, batch_size*2, num_workers = 0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images.to(CTX))                  # Generate predictions
        loss = F.cross_entropy(out, labels.to(torch.int64).cuda()) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images.to(CTX))                    # Generate predictions
        loss = F.cross_entropy(out, labels.to(torch.int64).cuda())   # Calculate loss
        acc = accuracy(out, labels.to(torch.int64).cuda())           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [5]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
#         print(x.shape())
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out
    
    
class Att_Net(ImageClassificationBase):   
    def __init__(self):
        super(Att_Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(300, 256, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(256),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            
            # Defining another 2D convolution layer
            Conv2d(256, 224, kernel_size=1, stride=1, padding=1),
            BatchNorm2d(224),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
            
            # Defining another 2D convolution layer
            Conv2d(224, 224, kernel_size=1, stride=1, padding=1),
            BatchNorm2d(224),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
            
            Conv2d(224, 200, kernel_size=1, stride=1, padding=1),
            BatchNorm2d(200),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
            
            Conv2d(200, 128, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
            
            Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
        )

        self.linear_layers = Sequential(
            Linear(128 * 26 * 7, 128*16),
            Linear(128 * 16, 128*4),
            Linear(128 * 4, 128*2),
            Linear(256, 64),
            Linear(64, 2),
        )

        self.attention = CBAM(gate_channels=128)
        self.softmax = Softmax()

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.attention(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
#         x = self.softmax(x)
        return x

In [6]:
# defining the model
model = Att_Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Att_Net(
  (cnn_layers): Sequential(
    (0): Conv2d(300, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(256, 224, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(224, 224, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (12): Conv2d(224, 200, kernel_size=(1, 1), stride=(1, 1), padding=(1, 1))
    (13): BatchNorm2d(200, eps=1e-05, momentum

In [7]:
from torchsummary import summary
summary(model, (300, 40, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 40, 3]         691,456
       BatchNorm2d-2           [-1, 256, 40, 3]             512
              ReLU-3           [-1, 256, 40, 3]               0
         MaxPool2d-4           [-1, 256, 20, 1]               0
            Conv2d-5           [-1, 224, 22, 3]          57,568
       BatchNorm2d-6           [-1, 224, 22, 3]             448
              ReLU-7           [-1, 224, 22, 3]               0
         MaxPool2d-8           [-1, 224, 22, 3]               0
            Conv2d-9           [-1, 224, 24, 5]          50,400
      BatchNorm2d-10           [-1, 224, 24, 5]             448
             ReLU-11           [-1, 224, 24, 5]               0
        MaxPool2d-12           [-1, 224, 24, 5]               0
           Conv2d-13           [-1, 200, 26, 7]          45,000
      BatchNorm2d-14           [-1, 200

C:\Users\Asus\anaconda3\envs\tf-btp\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [8]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [9]:
# CTX = torch.device('cuda')
# train_dl.to(CTX)  #train_dataset.train_data is a Tensor(input data)
# # train_dl.train_labels.to(CTX)
# # val_dl.train_data.to(CTX)  #train_dataset.train_data is a Tensor(input data)
# # val_dl.train_labels.to(CTX)
num_epochs = 5000
opt_func = torch.optim.Adam
lr = 0.001
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)


Epoch [0], train_loss: 1.0176, val_loss: 0.6911, val_acc: 0.5298
Epoch [1], train_loss: 0.6905, val_loss: 0.6888, val_acc: 0.5347
Epoch [2], train_loss: 0.6894, val_loss: 0.6840, val_acc: 0.5321
Epoch [3], train_loss: 0.6860, val_loss: 0.6930, val_acc: 0.5213
Epoch [4], train_loss: 0.6825, val_loss: 0.6839, val_acc: 0.5086
Epoch [5], train_loss: 0.6840, val_loss: 0.6818, val_acc: 0.4953
Epoch [6], train_loss: 0.6836, val_loss: 0.7043, val_acc: 0.5295
Epoch [7], train_loss: 0.6859, val_loss: 0.6854, val_acc: 0.4979
Epoch [8], train_loss: 0.6823, val_loss: 0.6821, val_acc: 0.5086
Epoch [9], train_loss: 0.6818, val_loss: 0.6888, val_acc: 0.5402
Epoch [10], train_loss: 0.6808, val_loss: 0.6877, val_acc: 0.5213
Epoch [11], train_loss: 0.6795, val_loss: 0.7008, val_acc: 0.5112
Epoch [12], train_loss: 0.6828, val_loss: 0.6891, val_acc: 0.5321
Epoch [13], train_loss: 0.6813, val_loss: 0.6940, val_acc: 0.5138
Epoch [14], train_loss: 0.6791, val_loss: 0.6850, val_acc: 0.5217
Epoch [15], train_lo

Epoch [124], train_loss: 0.0000, val_loss: 7.2775, val_acc: 0.5997
Epoch [125], train_loss: 0.0000, val_loss: 7.3251, val_acc: 0.5942
Epoch [126], train_loss: 0.0000, val_loss: 7.3685, val_acc: 0.5916
Epoch [127], train_loss: 0.0000, val_loss: 7.3124, val_acc: 0.6079
Epoch [128], train_loss: 0.0000, val_loss: 7.4094, val_acc: 0.5916
Epoch [129], train_loss: 0.0000, val_loss: 7.4194, val_acc: 0.5971
Epoch [130], train_loss: 0.0000, val_loss: 7.4585, val_acc: 0.6049
Epoch [131], train_loss: 0.0000, val_loss: 7.4987, val_acc: 0.5968
Epoch [132], train_loss: 0.0000, val_loss: 7.5555, val_acc: 0.6049
Epoch [133], train_loss: 0.0000, val_loss: 7.5311, val_acc: 0.5997
Epoch [134], train_loss: 0.0000, val_loss: 7.4634, val_acc: 0.6075
Epoch [135], train_loss: 0.0000, val_loss: 7.5769, val_acc: 0.6049
Epoch [136], train_loss: 0.0000, val_loss: 7.5913, val_acc: 0.5994
Epoch [137], train_loss: 0.0000, val_loss: 7.5780, val_acc: 0.6131
Epoch [138], train_loss: 0.0000, val_loss: 7.5938, val_acc: 0.

Epoch [247], train_loss: 0.0382, val_loss: 2.6138, val_acc: 0.5704
Epoch [248], train_loss: 0.0207, val_loss: 3.5619, val_acc: 0.5861
Epoch [249], train_loss: 0.0155, val_loss: 4.2620, val_acc: 0.5753
Epoch [250], train_loss: 0.0100, val_loss: 4.3079, val_acc: 0.5805
Epoch [251], train_loss: 0.0044, val_loss: 5.4982, val_acc: 0.5887
Epoch [252], train_loss: 0.0164, val_loss: 5.0975, val_acc: 0.6043
Epoch [253], train_loss: 0.0284, val_loss: 3.8379, val_acc: 0.5701
Epoch [254], train_loss: 0.0021, val_loss: 5.9253, val_acc: 0.5779
Epoch [255], train_loss: 0.0014, val_loss: 6.8023, val_acc: 0.5672
Epoch [256], train_loss: 0.0013, val_loss: 6.8669, val_acc: 0.5535
Epoch [257], train_loss: 0.0094, val_loss: 4.9455, val_acc: 0.5490
Epoch [258], train_loss: 0.0366, val_loss: 3.3485, val_acc: 0.6020
Epoch [259], train_loss: 0.0079, val_loss: 4.3745, val_acc: 0.5805
Epoch [260], train_loss: 0.0084, val_loss: 4.2365, val_acc: 0.5880
Epoch [261], train_loss: 0.0032, val_loss: 5.1688, val_acc: 0.

Epoch [370], train_loss: 0.0000, val_loss: 8.4841, val_acc: 0.5405
Epoch [371], train_loss: 0.0000, val_loss: 8.5271, val_acc: 0.5434
Epoch [372], train_loss: 0.0000, val_loss: 8.5678, val_acc: 0.5460
Epoch [373], train_loss: 0.0000, val_loss: 8.6224, val_acc: 0.5460
Epoch [374], train_loss: 0.0000, val_loss: 8.6827, val_acc: 0.5434
Epoch [375], train_loss: 0.0000, val_loss: 8.5969, val_acc: 0.5434
Epoch [376], train_loss: 0.0000, val_loss: 8.6679, val_acc: 0.5460
Epoch [377], train_loss: 0.0000, val_loss: 8.6490, val_acc: 0.5434
Epoch [378], train_loss: 0.0000, val_loss: 8.7261, val_acc: 0.5460
Epoch [379], train_loss: 0.0000, val_loss: 8.7458, val_acc: 0.5460
Epoch [380], train_loss: 0.0000, val_loss: 8.6858, val_acc: 0.5379
Epoch [381], train_loss: 0.0000, val_loss: 8.7018, val_acc: 0.5379
Epoch [382], train_loss: 0.0000, val_loss: 8.7398, val_acc: 0.5408
Epoch [383], train_loss: 0.0000, val_loss: 8.8745, val_acc: 0.5434
Epoch [384], train_loss: 0.0000, val_loss: 8.8479, val_acc: 0.

Epoch [492], train_loss: 0.0000, val_loss: 11.0377, val_acc: 0.5405
Epoch [493], train_loss: 0.0000, val_loss: 11.0565, val_acc: 0.5379
Epoch [494], train_loss: 0.0000, val_loss: 11.1914, val_acc: 0.5405
Epoch [495], train_loss: 0.0000, val_loss: 11.1693, val_acc: 0.5379
Epoch [496], train_loss: 0.0000, val_loss: 11.1649, val_acc: 0.5379
Epoch [497], train_loss: 0.0000, val_loss: 11.1341, val_acc: 0.5405
Epoch [498], train_loss: 0.0000, val_loss: 11.1509, val_acc: 0.5379
Epoch [499], train_loss: 0.0000, val_loss: 11.1881, val_acc: 0.5353
Epoch [500], train_loss: 0.0000, val_loss: 11.1833, val_acc: 0.5431
Epoch [501], train_loss: 0.0000, val_loss: 11.1419, val_acc: 0.5405
Epoch [502], train_loss: 0.0000, val_loss: 11.1817, val_acc: 0.5379
Epoch [503], train_loss: 0.0000, val_loss: 11.2564, val_acc: 0.5379
Epoch [504], train_loss: 0.0000, val_loss: 11.2027, val_acc: 0.5405
Epoch [505], train_loss: 0.0000, val_loss: 11.2618, val_acc: 0.5379
Epoch [506], train_loss: 0.0000, val_loss: 11.09

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

def plot_accuracies(history):
    """ Plot the history of accuracies"""
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    

plot_accuracies(history)

def plot_losses(history):
    """ Plot the losses in each epoch"""
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(history)

In [ ]:
max([x['val_acc'] for x in history])